<a href="https://colab.research.google.com/github/felipefreitassilva/Automatizando/blob/main/Trabalho01/Trabalho01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/EngenhariaSoftwarePUCRS/Inteligencia_Artificial/blob/develop/Trabalho01/Trabalho01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [230]:
integrantes = ["Augusto Baldino", "Felipe Freitas", "Isabela Kuser", "Luiza Heller", "Maria Eduarda Maia", "Paola Lopes"]
useful_links = [
    "https://pandas.pydata.org/pandas-docs/version/1.0/user_guide/style.html",
    "https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/",
    "https://www.geeksforgeeks.org/how-to-add-header-row-to-a-pandas-dataframe/",
    "https://discuss.streamlit.io/t/is-it-possible-to-center-data-on-cell-like-excel/32045/7",
]

In [231]:
import pandas as pd
from matplotlib import pyplot
from seaborn.palettes import mpl_palette as colors
from sklearn import model_selection, neighbors
from typing import Literal

In [232]:
targetClass = "Outcome"
columnNames = [
    "Top-Left", "Top-Middle", "Top-Right",
    "Middle-Left", "Middle-Middle", "Middle-Right",
    "Bottom-Left", "Bottom-Middle", "Bottom-Right",
    targetClass,
]
dataset = pd.read_csv(
    "data/tic-tac-toe.data",
    names=columnNames,
)
dataLinesCount = 958

In [233]:
X_VALUE = 1
B_VALUE = 0
O_VALUE = -1

def cell_to_number(cell: Literal['b', 'x', 'o']):
  if cell == 'b':
    return B_VALUE

  if cell == 'x':
    return X_VALUE

  if cell == 'o':
    return O_VALUE

  return cell

In [234]:
def o_has_won(board: list[str], index) -> bool:
    if len(board) != 9:
        raise ValueError("Board cannot have any different number of rows x columns than 9")

    # TODO: Check diagonal victories
    for i in range(0, 9, 3):
        linha = board[i:i+3]
        if all(cell == O_VALUE for cell in linha):
            return True

    return False

In [235]:
dataset.head(5)

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Outcome
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [236]:
for columnName in columnNames:
  dataset[columnName] = dataset[columnName].apply(cell_to_number)

In [239]:
dataset.head(5)

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Outcome
0,1,1,1,1,-1,-1,1,-1,-1,X Ganhou
1,1,1,1,1,-1,-1,-1,1,-1,X Ganhou
2,1,1,1,1,-1,-1,-1,-1,1,X Ganhou
3,1,1,1,1,-1,-1,-1,0,0,X Ganhou
4,1,1,1,1,-1,-1,0,-1,0,X Ganhou


In [237]:
# NOTE: Run only once
for index, row in dataset.iterrows():
    board = row.array.tolist()[:-1]

    if row[targetClass] == 'positive':
        targetValue = "X Ganhou"
    elif o_has_won(board, index):
        targetValue = "O Ganhou"
    elif B_VALUE in board:
        targetValue = "Tem Jogo"
    else:
        targetValue = "Deu Velha"

    dataset.at[index, targetClass] = targetValue

In [238]:
# Count the occurrences of each outcome to verify

x_ganhou_count = dataset[targetClass].eq('X Ganhou').sum()
print("'X Ganhou' outcomes:", x_ganhou_count)

o_ganhou_count = dataset[targetClass].eq('O Ganhou').sum()
print("'O Ganhou' outcomes:", o_ganhou_count)

deu_velha_count = dataset[targetClass].eq('Deu Velha').sum()
print("'Deu Velha' outcomes:", deu_velha_count)

tem_jogo_count = dataset[targetClass].eq('Tem Jogo').sum()
print("'Tem Jogo' outcomes:", tem_jogo_count)


Count of 'X Ganhou' outcomes: 626
Count of 'O Ganhou' outcomes: 108
Count of 'Deu Velha' outcomes: 16
Count of 'Tem Jogo' outcomes: 208


In [ ]:
# @title Win for X
dataset.groupby('Win for X').size().plot(kind='barh', color=colors('Dark2'))
pyplot.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
train_set, test_set = model_selection.train_test_split(dataset, train_size=0.8)

kNN_classifier = neighbors.KNeighborsClassifier(n_neighbors=5)

In [ ]:
train_set_x = train_set.drop(columns=[targetClass])
train_set_y = train_set[targetClass]

kNN_classifier.fit(train_set_x, train_set_y)

In [ ]:
test_set_x = test_set.drop(columns=[targetClass])
test_set_y = test_set[targetClass].values

predictions = kNN_classifier.predict(test_set_x)

success_count = 0
print('\t', "Esperado | Obtido")
for i in range(0, len(test_set_y)):
  if i == len(predictions):
    break
  if equals := (predictions[i] == test_set_y[i]):
    success_count += 1
    continue
  print("\033[31m", f"Erro na predição da linha {i}:", "\033[0m")
  print('\t', predictions[i], '|', test_set_y[i], '\n')

error_count = len(predictions) - success_count
if error_count == 0:
  # Remove print "Esperado | Obtido"
  print('\b' * len("\tEsperado | Obtido"))

print("Predições Corretas", '|', "Erros")
print('\t      ', success_count, '|', error_count)